# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from jupyter_client.manager import KernelManager
from IPython.display import display, Image
import base64
from pathlib import Path
from IPython.core.magic import register_line_magic, register_line_cell_magic

## Remote Kernel Backend

In [ ]:
#| export
#| notest
def set_ssh_config(port, user="", alias="remote_server_sshpyk", proxyname="bore.pub", config_path="~/.ssh/config"):
    config_path = Path(config_path).expanduser()
    if not config_path.exists(): config_path.touch()
    text = config_path.read_text()
    if f"Host {alias}" not in text: 
        assert user != "", "Must specify username when creating ~/.ssh/config info"
        block = f"""
Host {alias}
    HostName {proxyname}
    Port {port}
    User {user}
    BatchMode yes
    ControlMaster auto
    ControlPath ~/.ssh/sshpyk_%r@%h_%p
    ControlPersist 10m
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
"""
        config_path.write_text(text + block)
    else:
        lines = text.splitlines()
        in_target_block = False
        for i, line in enumerate(lines):
            if line.startswith("Host "):
                in_target_block = (line == f"Host {alias}")
            elif in_target_block and line.strip().startswith("Port "):
                lines[i] = f"    Port {port}"
            elif proxyname and in_target_block and line.strip().startswith("HostName "):
                lines[i] = f"    HostName {proxyname}"
            elif user and in_target_block and line.strip().startswith("User "):
                lines[i] = f"    User {user}"
        config_path.write_text("\n".join(lines) + "\n")

In [ ]:
#| export
_ipf_km, _ipf_kc = None, None            # "ipf" = "ipyfernel" ;-) 
def ipf_startup(kernel_name="chonk_python"):  
    "Start up the remote kernel"
    global _ipf_km, _ipf_kc 
    if _ipf_km is None and _ipf_kc is None: #only do this at startup
        _ipf_km = KernelManager(kernel_name=kernel_name)
        _ipf_km.start_kernel()
        _ipf_kc = _ipf_km.client()
        _ipf_kc.start_channels()
        _ipf_kc.wait_for_ready(timeout=30)
        print("Success: remote kernel started")
    else: 
        print("ipf_startup: already running")

In [ ]:
def output_hook(msg):
    "How to handle output from the remote kernel."
    mt = msg["msg_type"]
    content = msg.get("content", {})
    if mt == "stream":
        print(content["text"], end="", flush=True)
    elif mt == "error":
        print('\n'.join(content.get("traceback", [])))
    elif mt in ("display_data", "update_display_data"):
        data = content.get("data", {})
        if "image/png" in data:
            display(Image(base64.b64decode(data["image/png"])))
        elif "text/plain" in data:
            print(data["text/plain"])

In [ ]:
def ipf_exec(code, verbose=False):
    "Execute code on the remote kernel." 
    assert _ipf_kc is not None, "ipf_exec: need to run ipf_startup() first"
    result = _ipf_kc.execute_interactive(code=code, output_hook=output_hook)
    _ipf_kc.last_result = result  # stash it for optional inspection later
    if verbose: return result

In [ ]:
def ipf_shutdown():
    "Terminates the remote kernel"
    global _ipf_km, _ipf_kc
    try:
        if _ipf_kc is not None: _ipf_kc.stop_channels()
        if _ipf_km is not None: _ipf_km.shutdown_kernel(now=True)  # 'now=True' forces immediate shutdown
    except: pass  # Don't hang on errors
    _ipf_km, _ipf_kc = None, None

In [ ]:
def _execute_remotely(lines):
    "Take commands from magics and send to ipf_exec"
    code = ''.join(lines)
    if 'get_ipython()' in code: return lines  # let solveit internals pass through
    # Make sure our magics execute locally
    if code.strip().startswith(('%local', '%%local', '%unset_remote', '%resume_remote', '%set_remote')):
        return lines
    return [f"ipf_exec({repr(code)})\n"]

In [ ]:
#| export
@register_line_cell_magic
def local(line, cell=None):
    "local execution: works as %local and as %%local"
    get_ipython().run_cell(cell if cell else line) 

In [ ]:
#| hide

#import nbdev; nbdev.nbdev_export()
!nbdev_export --procs scrub_magics